In [1]:
import json
import grequests
import requests
import os
import pandas as pd
import time
from pvlib.forecast import HRRR
import datetime
import pytz

request_headers = {'Accept': 'application/json',
                   'Accept-Language': 'en-US'} 

class weather_forecaster_singlelocation(object):
    '''
    This class  gathers the weather forecasts at one station on a specified frequency. By
    default it uses pvlib to reference NOAA's HRRR forecast model, and returns the temperature and
    solar irradiation values. It reads a configuration file that specifies the station and sampling
    frequency. It's possible to use the Advanced Windows Testbed by specifying AWTB = True. Doing so
    turns off other station locations.
    '''
    
    def __init__(self, config, coordinates = False, AWTB=False, horizon=48, model=HRRR, irrad_vars=['ghi', 'dni', 'dhi']):
        '''
        Reads the config information and finds latitude, longitude, and timezone for the specified location.
        
        Inputs:
        config: The configuration file specifying the target location, sampling frequency, and output file.
        coordinates: Specifies that the user provided a config file with specific location coordinates
                     instead of a weather.gov.
        AWTB: Specifies whether to use the Advanced Windows Testbed or not.
        horizon: Desired length of the forecast. As of Nov 2, 2021 HRRR can provide up to 48 hours
                 but pvlib only returns 16.
        model: The desired NOAA forecasting tool.
        irrad_vars: The solar irradiation values to include in the forecast.
        '''
        
        # Set the forecast information
        self.irrad_vars = irrad_vars
        self.horizon = horizon
        self.forecaster = model()
        
        # Store configuration information
        self.config = config
        
        # If using the Advanced Windows Testbed use the corresponding coordinates and timezone
        if AWTB:
            self.location = '71t'
            self.coordinates = [-122.2501, 37.8788]
            self.tz = pytz.timezone("Etc/GMT+8")
        
        # If using coordinates read them from coordinates and timezone from the configuration data
        elif coordinates:
            self.location = self.config['location']
            self.coordinates = config['coordinates']
            self.tz = pytz.timezone(config['timezone'])
            
        # If specifying location via a weather.gov weather station
        else:
            self.location = self.config['location']

            # Get coordinates & timezone for station, store them in attributes
            url = 'https://api.weather.gov/stations/{}'.format(self.location)
            grequest = [grequests.get(url, verify=requests.certs.where(), headers=request_headers, timeout = 5)]
            gresponse = grequests.map(grequest)[0]
            
            if gresponse is None: # If there's no response from weather.gov
                print('Could not obtain station details. Will continue trying every 5 minutes')
                # Try to obtain station details until successful
                while gresponse is None:
                    import time
                    time.sleep(5*60)
                    grequest = [grequests.get(url, verify=requests.certs.where(), headers=request_headers, timeout = 5)]
                    gresponse = grequests.map(grequest)[0]
            
            response = json.loads(gresponse.content)
            self.coordinates = response['geometry']['coordinates']
            self.tz = response['properties']['timeZone']        

    def get_forecasts(self, now_str = 'N/A', start = None, end = None, save = False, return_result = False, json_output = False):
        '''
        Gathers forecasts for the specified station. Returns either the forecast or an
        error message as appropriate
        
        Inputs:
        now_str: String representation of the time that this function is called.
        start: User-specified start time for the forecast.
        end: User specified end time for the forecast.
        save: State whether or not to save the forecast as a .csv file.
        return_result: State whether or not to return the resulting forecast. By default
                       returns a pandas dataframe.
        json_output: Converts the output from a pandas dataframe to json.
        '''

        # Gather information about the location
        station = self.location
        lon = self.coordinates[0]
        lat = self.coordinates[1]
        tz = self.tz
            
        # Set forecast start and end times
        if not start:
            start_dt = pd.Timestamp(datetime.datetime.now().replace(minute=0, second=0, microsecond = 0), tzinfo = tz)
        else:
            start_dt = pd.Timestamp(start, tz=tz)
        if end:
            end_dt = end
        else:
            end_dt = start_dt + pd.Timedelta(hours=self.horizon)        

        # Forecast processing
        expected_index = pd.date_range(start_dt, end_dt, freq = 'H')
        data = pd.DataFrame(index = expected_index, columns = ['temp_air', 'wind_speed', 'ghi', 
                                                               'dni', 'dhi', 'total_clouds',
                                                               'low_clouds', 'mid_clouds', 
                                                               'high_clouds'])
        error_msg = None

        try:
            data = self.forecaster.get_processed_data(lat, lon, start_dt, end_dt)

        except requests.exceptions.ConnectionError as e:
            error_msg = 'ERROR: Lost connection to weather.gov. {}'.format(e)
        except requests.exceptions.HTTPError as e:
            error_msg = 'ERROR: An HTTP error occured. {}'.format(e)
        except requests.exceptions.URLRequired as e:
            error_msg = 'ERROR: Invalid URL supplied. {}'.format(e)
        except requests.exceptions.TooManyRedirects as e:
            error_msg = 'ERROR: Forecast request was redirected too many times. {}'.format(e)
        except requests.exceptions.ConnectTimeout as e:
            error_msg = 'ERROR: Timed out while connecting to weather.gov. {}'.format(e)
        except requests.exceptions.ReadTimeout as e:
            error_msg = 'ERROR: The weather.gov server did not return data in allowed time. {}'.format(e)
        except requests.exceptions.Timeout as e:
            error_msg = 'ERROR: The weather forecast request timed out. {}'.format(e)
        except Exception as e:
            error_msg = 'ERROR: An unexpected error occurred - {}'.format(e)

        # Save the forecast information to a .csv file if specified
        if save == True:
            folder_output = os.path.join(root, station)
            if not os.path.exists(folder_output):
                os.mkdir(folder_output)                
                    
            path_output = os.path.join(folder_output, 'Forecast_{}_{}.csv'.format(self.location, now_str))
            data.to_csv(path_output)
            
        # Return the forecast dictionary if specified
        if return_result == True:
            if json_output:
                data.index = data.index.tz_localize(None)
                return data.to_json(), error_msg
            else:
                return data, error_msg        
            
if __name__ == '__main__':
    
    # Run both _singlelocation and _multiplelocations one time
    try:
        root = os.path.dirname(os.path.realpath(__file__))
    except:
        root = os.getcwd()
        
    # Single location
    path_config = os.path.join(root, 'forecasts_config_singlelocation')
    config = json.load(open(path_config))

    config = {
        "location": "testing",
        "coordinates": [-73.2121, 44.4759],
        "timezone": "US/Eastern",
        "poll_interval": 60,
        "output_file": "forecast.csv"
    }
    
    forecaster = weather_forecaster_singlelocation(config, coordinates = True)
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M")
    result, error_msg = forecaster.get_forecasts(now_str=now, save=False, return_result=True, json_output=False)  
    #print('\n\nsingle forecast')
    #print(result)
    print("\nthe end")

/Users/thomasdefauw/miniconda3/envs/ada/lib/python3.8/site-packages/pvlib/forecast.py:20: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  warnings.warn(


test point 1

the end


In [2]:
#!pip install pvlib==0.9.0
#!pip install netCDF4==1.5.8